In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
custom_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(30),  # Randomly rotate the image by up to 30 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, saturation, and hue
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Randomly translate the image
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # Normalize the image
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=custom_transform,
)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]),
)

val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

100%|██████████| 344862509/344862509 [00:01<00:00, 226894336.33it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 1723028.32it/s]


100%|██████████| 14989/14989 [00:00<00:00, 15831886.84it/s]


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 28 * 28, 1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 102)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 256 * 28 * 28)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)


model = NeuralNetwork().to(device)


In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


In [ ]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    running_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def val_loop(dataloader, model, loss_fn, device):
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    return val_loss, correct * 100


In [ ]:
# Training
epochs = 100
start = time.time()
best_val_loss = float('inf')
patience = 3
counter = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer, device)
    val_loss, val_accuracy = val_loop(val_dataloader, model, loss_fn, device)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping")
            break

end = time.time()
print(f"Training took: {(end - start) / 60:.2f} minutes!")

Epoch 1/100
Train Loss: 4.6663, Val Loss: 4.5358, Val Accuracy: 4.22
Epoch 2/100
Train Loss: 4.4773, Val Loss: 4.3302, Val Accuracy: 8.73
Epoch 3/100
Train Loss: 4.3505, Val Loss: 4.1672, Val Accuracy: 9.90
Epoch 4/100
Train Loss: 4.1992, Val Loss: 4.0418, Val Accuracy: 15.20
Epoch 5/100
Train Loss: 4.0809, Val Loss: 3.9295, Val Accuracy: 19.51
Epoch 6/100
Train Loss: 3.9715, Val Loss: 3.8014, Val Accuracy: 20.78
Epoch 7/100
Train Loss: 3.8758, Val Loss: 3.7651, Val Accuracy: 21.47
Epoch 8/100
Train Loss: 3.7901, Val Loss: 3.6083, Val Accuracy: 25.20
Epoch 9/100
Train Loss: 3.7598, Val Loss: 3.5622, Val Accuracy: 24.61
Epoch 10/100
Train Loss: 3.6381, Val Loss: 3.5658, Val Accuracy: 25.98
Epoch 11/100
Train Loss: 3.5755, Val Loss: 3.4257, Val Accuracy: 27.06
Epoch 12/100
Train Loss: 3.5350, Val Loss: 3.3941, Val Accuracy: 27.75
Epoch 13/100
Train Loss: 3.4313, Val Loss: 3.3517, Val Accuracy: 28.92
Epoch 14/100
Train Loss: 3.3942, Val Loss: 3.2832, Val Accuracy: 29.02
Epoch 15/100
Train